In [ ]:
import bluepy
# prefperi_left = 'F8:A6:1C:CB:ED:A6'
# prefperi_right = 'F6:B2:2F:08:B9:48'
prefperi_custom = 'de:9c:60:51:f5:bf'
ss = bluepy.btle.Peripheral()
ss.connect(prefperi_custom,'random')

In [ ]:
import binascii
import numpy as np
from collections import deque
from time import sleep
values = deque()

                    
gestures = {
    '0 z, 0 y':    [0, 0], #0 z, 0 y
    '0 z, -45 y':  [0, -45], #0 z, -45 y
    '-45 z, 0 y':  [-45, 0], #Rotate thumb hand open (target thumb)
    '0 z, 30 y':   [0, 30], #Rotate thumb hand open (target thumb)
}

                #1 s1 th s2
result = {
    '0 z, 0 y': [ 1, 127, 275, 132], #0 z, 0 y
    '0 z, -45 y':  [ 1, 127, 270, 132], #0 z, -45 y
    '-45 z, 0 y': [ 1, 133, 288, 137], #Rotate thumb hand open (target thumb)
    '0 z, 30 y': [ 1, 124, 276, 132], #Rotate thumb hand open (target thumb)
}


# handle = i.getHandle()
# if handle == 12 or handle == 55:
#     continue
for i in gestures:
    print('get ready for ' + i)
    sleep(4)
    value = ss.readCharacteristic(16)
    decimalValue = (binascii.b2a_hex(value))
    splitted = [decimalValue[j:j+4] for j in range(0, len(decimalValue),4)]
    val = np.array(list(map(lambda x: int((x),16)/10, splitted)))
    idx = np.nonzero(val)
    val = val[idx][:-1]
    val = np.insert(val, 0, 1)
#     print('splayL thumb splayR')
    print(val[0:4])
    result[i] = val[0:4]
#     print('2 seconds')
#     sleep(2)

In [ ]:
import numpy as np
from sklearn import linear_model

gestures = [
    [0, 0], #0 z, 0 y
    [0, -45], #0 z, -45 y
    [-45, 0], #Rotate thumb hand open (target thumb)
    [0, 30] #Rotate thumb hand open (target thumb)
]

                #1 s1 th s2
result = [
    [1, 125, 133, 272], #0 z, 0 y
    [1, 125, 131, 267], #0 z, -45 y
    [1, 139, 141, 297], #Rotate thumb hand open (target thumb)
    [1, 134, 141, 296] #Rotate thumb hand open (target thumb)
]

theta = [] # np.zeros((len(gestures[0]), 3))

def getColumnY(Array, index):
    col = np.zeros(len(Array))

    for i in range(0, len(Array)):
        col[i] = Array[i][index]

    return col

def getColumnRes(Array, index):
    col = np.zeros((len(Array), 3))
    for i in range(0, len(Array)):
        col[i] = [x for x in Array[i][index:index+3]]

    return col

def ApplyTransformation(inputt, theta):
    result = np.zeros(len(theta))
    for i in range(len(theta)):
        result[i] = (inputt[i] * theta[i][0]) + theta[i][1]
    return result

for i in range(0, len(gestures[0])):
    y = getColumnY(gestures, i)
    x = getColumnRes(result, i)
    print(x.shape, y.shape)
#     x = np.array([x])
    regr = linear_model.LinearRegression()
    regr.fit(x,y)
    theta.append([x for x in regr.coef_])
    theta[i].append(regr.intercept_)

print(theta)
    


In [ ]:
regr.predict(x)

In [ ]:
thet = [2.72727273,  131.59090909,  -43.63636364, -5973.4090909090701]
res = [134,  141,  296, 1]
out = np.dot(thet, res)
out

In [ ]:
from sklearn.linear_model import LinearRegression
from joblib import dump, load
regr = LinearRegression()
X = np.array([[1, 1, 2], [1, 2, 2], [2, 2, 4], [2, 3, 5]])
# y = np.dot(X, np.array([1, 2, 3]))
y = np.array([1, 2, 3, 4])
print(y)
regr.fit(X,y)
regr.coef_
dump(regr, 'ss.joblib')

In [ ]:
load('ss.joblib')

In [ ]:
ss.disconnect()

In [ ]:
arr = np.array([1,2,3])
arr.tolist()

In [ ]:
TrainingName = ["close fist", "thumb up", "Thumb up, pinky out", "Open hand", "tuck thumb", "tuck thumb in fist", "ok sign", "touch middle finger",
                "touch middle finger and ring finger", "Spread hand open (target thumb)", "Rotate thumb hand open (target thumb)","Rotate thumb hand open (target thumb)", "Rotate thumb hand open (target thumb)",
                "Rotate thumb hand open (target thumb)", "Rotate thumb hand open (target thumb)", "Thumb over fist"]
len(TrainingName)

In [ ]:
TrainingY = [
  [ 0, 0, 0, -15, -90, -90, -90, -90], #Close fist
  [ 0, -15, -60, 0, -90, -90, -90, -90], #Thumb up
  [ 0, -15, -60, 0, -90, -90, -90, 0], #Thumb up, pinky out
  [ 0, -15, -60, 0, 0, 0, 0, 0], #Open hand
  [ 0, -15, 0, 40, 0, 0, 0, 0], #Tuck thumb
  [ 0, -15, 0, 40, -80, -80, -80, -80], #Tuck thumb in fist
  [ -35, 55, -65, 30, -55, 0, 0, 0], #ok sign
  [ -60, 33, -35, 30, 0, -60, 0, 0], #touch middle finger
  [ -60, 33, -35, 30, 0, -60, -60, 0], #touch middle finger and ring finger
  [ 0, -15, -60, 0, 0, 0, 0, 0], #Spread hand open (target thumb)
  [ 0, 30, -60, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 60, -60, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 60, -45, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 15, -15, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 0, 0, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ -15, 90, -45, 0, -90, -90, -90, -90], #Thumb over fist
]

len(TrainingY)

In [ ]:
'renewing recording for %s in %f' % ('lv', 5.6)

In [ ]:
import binascii
import numpy as np

for i in ss.getCharacteristics():  
    try:
        if i.valHandle in range(16,42):
            print(i.uuid)
            handle = i.valHandle
            print('handle ', handle)
            value = ss.readCharacteristic(handle)
            decimalValue = (binascii.b2a_hex(value))
            print(decimalValue)
            splitted = [decimalValue[i:i+4] for i in range(0, len(decimalValue),4)]
            val = np.array(list(map(lambda x: int((x),16), splitted)))

            print(val)
            print('\n')
    except:
        pass   
    



In [ ]:
value = b"\x5a"
ss.writeCharacteristic(29, value)
res = ss.readCharacteristic(29)
decimalValue = (binascii.b2a_hex(res))
decimalValue = int(decimalValue,16)
decimalValue

In [ ]:
svc = ss.getServiceByUUID('00001701-7374-7265-7563-6873656e7365')
char = svc.getCharacteristics()[0]
handle = char.valHandle
ss.writeCharacteristic(handle+1, b'\x01\x00')

In [ ]:
res = ss.readCharacteristic(17)
decimalValue = (binascii.b2a_hex(res))
decimalValue

In [ ]:
p.disconnect()

In [ ]:
res = p.waitForNotifications(0.9)

In [ ]:
data_test = np.array([0,1,2,3,4,5,6,7])
data_test2 = np.array([100,200])
conc = np.concatenate((90 + data_test[-4:], data_test2[1:2]))
conc

In [ ]:
from math import log
import matplotlib.pyplot as plt
x = np.arange(45,75, 0.5)/55
y = [log(elem,(1.18)) for elem in x]
plt.plot(x,y)

In [ ]:
TrainingY = [
  [ 0, 0, 0, -15, -90, -90, -90, -90], #Close fist
  [ 0, -15, -60, 0, -90, -90, -90, -90], #Thumb up
  [ 0, -15, -60, 0, -90, -90, -90, 0], #Thumb up, pinky out
  [ 0, -15, -60, 0, 0, 0, 0, 0], #Open hand
  [ 0, -15, 0, 40, 0, 0, 0, 0], #Tuck thumb
  [ 0, -15, 0, 40, -80, -80, -80, -80], #Tuck thumb in fist
  [ -35, 55, -65, 30, -55, 0, 0, 0], #ok sign
  [ -60, 33, -35, 30, 0, -60, 0, 0], #touch middle finger
  [ -60, 33, -35, 30, 0, -60, -60, 0], #touch middle finger and ring finger
  [ 0, -15, -60, 0, 0, 0, 0, 0], #Spread hand open (target thumb)
  [ 0, 30, -60, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 60, -60, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 60, -45, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 15, -15, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ 0, 0, 0, 0, 0, 0, 0, 0], #Rotate thumb hand open (target thumb)
  [ -15, 90, -45, 0, -90, -90, -90, -90], #Thumb over fist
]

thumb = []
for i in TrainingY:
    thumb.append(i[0:4])

In [ ]:
import numpy as np

gestures = [
    [0, 0], #0 z, 0 y
    [0, -45], #0 z, -45 y
    [-45, 0], #Rotate thumb hand open (target thumb)
    [0, 30] #Rotate thumb hand open (target thumb)
]

                #1 s1 th s2
result = [
    [1, 125, 133, 272], #0 z, 0 y
    [1, 125, 131, 267], #0 z, -45 y
    [1, 139, 141, 297], #Rotate thumb hand open (target thumb)
    [1, 134, 141, 296] #Rotate thumb hand open (target thumb)
]

ones = np.array([
    [1, 1, 1, 0],
    [0, 1, 1, 1]
])

idx = np.nonzero(ones)
ones[idx]

In [ ]:
theta = np.array([])
theta = np.append(theta, [6])
theta

In [ ]:
activeSensors = np.array([
    [1, 1, 0, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1]
])

arr = np.arange(10,18)

def getColumnX(Array):
    result = np.zeros((len(Array), 3))
    for index in range(len(activeSensors)):
        idx = np.nonzero(activeSensors[index])
        list = [x for x in Array[idx]]
        for r in range(len(list)):
            result[index][r] = list[r]
        print(len(list))
    
    return result
  
getColumnX(arr)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
y

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LogisticRegression
import rospy, sys
from stretchsense.msg import ssCap
from collections import deque
from joblib import load, dump
from time import sleep
from threading import Thread
import numpy as np

# fingGesture = {
#     'halt': 0,
#     'one': 1,
#     'two': 2,
#     'three': 3,
#     'four': 4,
#     'five': 5
# }

fingGesture = {
    'down': 0,
    'up': 1
}

rospy.init_node('readSSCap', anonymous=True)
xGestures = np.zeros((len(fingGesture)*100, 2))
yGestures = np.zeros(len(fingGesture)*100)
count = 0
idx = 0
change = deque((0,0), maxlen=2)
stop_thread = False

def get_key(val): 
    for key, value in fingGesture.items(): 
         if val == value: 
             return key 

def ssCallback(data):
    global count, idx
    gest = [x for x in data.values[0:2]]
#     predict(gest)
    
    if idx == len(fingGesture):
        print('stopping')
        subStop()

    else:
        xGestures[count] = gest
        yGestures[count] = int(count/100)
        idx = int((count+1)/100)
        count = count + 1
        change.append(int(count/100))

    if np.std(change)>0 and idx < len(fingGesture):
        print(get_key(idx))
        sleep(5)
        pass
    
def ssSub():
    rospy.Subscriber('stretchSenseCap_10Channel', ssCap, ssCallback)
    rospy.spin()

sub_thread = Thread(target=ssSub)
sub_thread.start()

def subStop():
    sub_thread.join()


In [ ]:
model = load('fingGest.joblib')

def predict(gest):
    res = model.predict([gest]) 
    g = get_key(res[0])
    sys.stdout.write("\r %s__" % g)
    sys.stdout.flush()
    
ssSub()

In [ ]:
lr = LogisticRegression(random_state=0).fit(xGestures, yGestures)
lr.score(xGestures, yGestures)
dump(lr, 'fingGest.joblib')

In [ ]:
d = [1.70111838, 1.58879359, 0.53324571, 0.62825394]
d = [round(x/1.57) for x in d]
d

In [ ]:
zip(cc[0], cc[1])